In [2]:
import pickle
import pandas as pd

In [3]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

In [4]:
categorical = ['PULocationID', 'DOLocationID']


In [5]:

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df



In [13]:
year = 2022
month = 2
taxi_type = 'yellow'

In [6]:
df = read_data(f'https://d37ci6vzurychx.cloudfront.net/trip-data/{taxi_type}_tripdata_{year:04d}-{month:02d}.parquet')
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

In [11]:
import numpy as np
np.std(y_pred)

5.28140357655334

In [22]:
import pyarrow
import os

df_result = pd.DataFrame()
df_result['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')
df_result['prediction'] = y_pred

output_path = f"output/{taxi_type}/{year:04d}"
if not os.path.exists(output_path):
    os.makedirs(output_path)

output_file = f'{output_path}/{month:02d}.parquet'

In [23]:

df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)